In [3]:
# %pip install --upgrade --quiet  langchain-google-genai pillow

In [4]:
# !pip install sentence-transformers==2.2.2

In [5]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader   #Generic loaded to load any code base through langchain
from langchain.document_loaders.parsers import  LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface_hub import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import  ChatGooglePalm
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
# from sentence_transformers import HuggingFaceHubEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
## Clone Github Repos
%pwd

'c:\\Users\\nagan\\OneDrive\\Desktop\\Ineuron_Gen_AI_Course\\Projects\\In_Class_Projects\\Source-Code-Analyzer-Using-GenAI\\research'

In [7]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [20]:
Repo.clone_from('https://github.com/entbappy/End-to-end-ML-Project-Implementation.git',to_path='test_repo/')

In [8]:
# Loading the Repository
repo_path='test_repo/'
loader=GenericLoader.from_filesystem(
         path=repo_path+ 'src/mlProject',
         glob="**/*",
         suffixes=[".py"],
         parser=LanguageParser(language=Language.PYTHON,parser_threshold=500 ),
         show_progress=True,
     )

In [9]:
document=loader.load()

c:\Users\nagan\anaconda3\envs\llmapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 21/21 [00:00<00:00, 2334.68it/s]


In [10]:
document

[Document(page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")', metadata={'source': 'test_repo\\src\\mlProject\\__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os\nimport urllib.request as request\nimport zipfile\nfrom mlProject import logger\nfrom mlProject.utils.common import get_size\nfrom mlProject.entity.config_entity import DataIngestionConfig\nfrom pathlib import Path\n\n\nclass DataIngestion:\n    def __init__(self, config: DataIngestionConfig):\n        self.config = config\n\n    \n    def download_file(self):\n    

In [11]:
## Context Aware Splitting-Chunkings
document_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=1500,
                                                               chunk_overlap=200)

In [12]:
text=document_splitter.split_documents(document)

In [13]:
len(text)

20

Embedding Model


In [14]:
Hugging_Face_API_Key='*********************************'


In [15]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### Knowledge Base (Vecor DB)

In [17]:
chroma_vector_db=Chroma.from_documents(text,embedding=embeddings,persist_directory='./data')        #persist_directory will create a DB with data
chroma_vector_db.persist()

### LLM Wrapper

In [18]:
# # google model 
# !pip install google.generativeai

In [19]:
GEMINI_API_KEY='************'

In [20]:
LLM=ChatGooglePalm(google_api_key=GEMINI_API_KEY,model="gemini-pro")

In [21]:
# Defining memory for the chat history-Buffer Memory
memory=ConversationSummaryMemory(llm=LLM,memory_key='chat_history',return_messages=True)

In [22]:
# QA Chain or Wrapper
qa_chain=ConversationalRetrievalChain.from_llm(LLM,retriever=chroma_vector_db.as_retriever(search_type="similarity",search_kwargs={"k":3}),memory=memory)

### Q&A

In [23]:
question="What is Model Trainer Stage"

In [24]:
result=qa_chain(question)
print(result['answer'])

The Model Trainer Stage is a stage in the machine learning pipeline that is responsible for training the model. The model trainer takes the model definition and the training data and uses it to train the model. The model trainer can be implemented in a variety of ways, but it typically involves using a machine learning algorithm to learn the relationship between the input data and the output labels.

The Model Trainer Stage is a critical stage in the machine learning pipeline, as it is responsible for producing the trained model that will be used to make predictions. The quality of the trained model is dependent on the quality of the model definition, the training data, and the model trainer algorithm.

There are a number of different model trainer algorithms that can be used, each with its own strengths and weaknesses. Some of the most popular model trainer algorithms include:

* **Supervised learning:** Supervised learning algorithms are trained on labeled data, which means that the 

In [25]:
question_2="What is Data Ingestion Class"

In [26]:
result_2=qa_chain(question_2)
print(result_2['answer'])

The Data Ingestion Class in the Model Trainer Stage is responsible for loading the data from the source and preparing it for training. This includes tasks such as cleaning and formatting the data, and splitting it into training and test sets. The Data Ingestion Class also provides a number of features that make it easy to work with different types of data, such as support for CSV files, JSON files, and SQL databases.

The Data Ingestion Class is a key component of the Model Trainer Stage, and it is essential for ensuring that the data is ready for training. The Data Ingestion Class is also a flexible and powerful tool that can be used to work with a variety of different types of data.

Here is an example of how the Data Ingestion Class can be used to load data from a CSV file:

```
import mlProject.components.data_ingestion as data_ingestion

# Load the data from the CSV file
data = data_ingestion.load_csv("data.csv")

# Print the first few rows of the data
print(data[:5])
```

This co